In [1]:
import pandas as pd
import numpy as np
import sklearn.cluster as skc
import sklearn.preprocessing as skp
from collections import defaultdict

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

## Read in Offline Nearest Neighbor processed data 

In [2]:
data_keys = pd.read_csv('../Processing/FrameContainer/DataRows/Data_keys.csv', header=None)
data_keys = data_keys.iloc[:,0].tolist()

In [ ]:
# for idx in range(0,15):
#     print("Starting recombination for Query Chunk: %d" % idx)
#     path = '../Processing/FrameContainer/QueryRows/'
#     filename = 'Query_Keys_'
#     currentKeys = pd.read(path + filename + idx + '.csv')
    

## Write our files for offline processing

In [ ]:
## Pairwise DTF of all "data-rows" (10000 rows)

In [ ]:
# n_jobs = -1 to turn on parallel
start = timer()
K = skm.pairwise_kernels(dataRows_df, metric=dtw, n_jobs=1)
end = timer()
print(end - start)      

np.savetxt("Processing/DTW_Matrix.out.gz", K, delimiter=',')

# FASTDTW ON SCALE 4E-4 PER PAIR
# AT 10^3 SAMPLES, 10^6 ELS, SO 10^2 SECONDS. Tolerance for production up to 10^4 elements 
# several hours of runtime

In [ ]:
np.savetxt("Processing/DTW_Matrix.out.gz", K, delimiter=',')

In [4]:
K = np.loadtxt('../Processing/DTW_Matrix.out.gz', delimiter=',')

## Hierarchical Clustering

In [5]:
## Run Hierarchical clustering on the matrix 
cluster_ward = skc.AgglomerativeClustering(n_clusters=10, affinity='euclidean', connectivity=K, compute_full_tree=True, linkage='ward')
# cluster_complete = skc.AgglomerativeClustering(n_clusters=10, affinity='euclidean', connectivity=K, compute_full_tree=True, linkage='complete')
# cluster_average = skc.AgglomerativeClustering(n_clusters=10, affinity='euclidean', connectivity=K, compute_full_tree=True, linkage='average')

# Predict the class labels
labels_ward = cluster_ward.fit_predict(K)
# labels_complete = cluster_complete.fit_predict(K)
# labels_average = cluster_average.fit_predict(K)

## Read-in Offline Processed Files

In [11]:
# Helpful dict structure
labeled_data = dict(list(zip(data_keys, labels_ward)))

In [6]:
dataResult_PATH = '../Processing/FrameContainer/DataRows/Data_Keys.csv'
data_keys = pd.read_csv(dataResult_PATH, delimiter='\n', header=None)
data_keys = data_keys[0].values.tolist()

for idx in range(0,15):
    print("Starting join process for Query Chunk: %d" % idx)
    
    outResult_PATH = '../Processing/FrameContainer/OutRows/'
    queryKey_PATH = '../Processing/FrameContainer/QueryRows/'
    
    out_FNAME = 'Out_Rows_'
    query_FNAME = 'Query_Keys_'
    
    queryIndices = pd.read_csv(queryKey_PATH + query_FNAME + str(idx) + '.csv', delimiter='\n', header=None)
    searchResults = pd.read_csv(outResult_PATH + out_FNAME + str(idx) + '.txt', delimiter='\n', header=None)
    
    queryIndices = queryIndices[0].values.tolist()
    searchResults = searchResults[0].values.tolist()
    # Map search results to corresponding data row index
    mappedResults = [data_keys[i] for i in searchResults]
    
    for idx, q in enumerate(queryIndices):
        labeled_data[q] = labeled_data[mappedResults[idx]]

class_data = defaultdict(list)
for k, v in labeled_data.items():
    class_data[v].append(k)

Starting join process for Query Chunk: 0


NameError: name 'labeled_data' is not defined

In [24]:
for k in class_data.keys():
    print("Label %d | Class size: %d" % (k, len(class_data[k])))

Label 0 | Class size: 982
Label 1 | Class size: 5908
Label 2 | Class size: 2915
Label 3 | Class size: 2991
Label 4 | Class size: 21433
Label 5 | Class size: 20520
Label 6 | Class size: 89502
Label 7 | Class size: 247
Label 8 | Class size: 186
Label 9 | Class size: 379


## OPTIONAL TRAINING ON CLASSES

### Train on Class 9 - 379 Examples

In [26]:
rawData_df = pd.read_csv('../input/train_1.csv')

In [9]:
FIXED_ROW_CAP = 10e3

In [8]:
FIXED_ROW_CAP

100000.0

In [5]:
CHECK_LABEL = 6
r1 = class_data[CHECK_LABEL][12]
r2 = class_data[CHECK_LABEL][40]

r1dat = skp.MinMaxScaler().fit_transform(rawData_df.ix[r1,1:].values)
r2dat = skp.MinMaxScaler().fit_transform(rawData_df.ix[r2,1:].values)

first = pd.Series(r1dat, index=rawData_df.columns[1:])
first.plot()

second = pd.Series(r2dat, index=rawData_df.columns[1:])
second.plot()

NameError: name 'class_data' is not defined

In [ ]:
for gap in range(0,60):
    